In [26]:
from FINData import FINData
import vectorbtpro as vbt
import numpy as np
import pandas as pd

# 1. Lấy dữ liệu HPG từ FINData
data = FINData.pull(
    ['HPG'],
    start='2022-01-01'
)

# Giả định: data là dict, trong đó data["Close"] là DataFrame index=Date, columns=ticker
close_df = data.get("Close")  # <--- nếu tên key khác (ví dụ 'close'), sửa tại đây

# Nếu close_df có nhiều cột (nhiều ticker), lấy đúng HPG
if isinstance(close_df, pd.DataFrame):
    close = close_df['HPG']
else:
    # Trường hợp FINData trả luôn Series cho HPG
    close = close_df

# Dọn NaN
close = close.dropna()


In [27]:
print(close.head())
print(close.shape)


DateTime
2022-01-04 00:00:00+00:00    26.80
2022-01-05 00:00:00+00:00    26.83
2022-01-06 00:00:00+00:00    26.43
2022-01-07 00:00:00+00:00    26.26
2022-01-10 00:00:00+00:00    26.17
Name: HPG, dtype: float64
(973,)


In [29]:
# 2.1. Tính RSI
rsi = vbt.RSI.run(close, window=14)

# 2.2. Tạo entries & exits
# Entry khi RSI < 30
entries_rsi = rsi.rsi_crossed_below(30)  # SeriesFrame (1 cột)
# Exit khi RSI > 70
exits_rsi = rsi.rsi_crossed_above(70)

# 2.3. Backtest với Portfolio.from_signals
pf_rsi = vbt.Portfolio.from_signals(
    close,
    entries=entries_rsi,
    exits=exits_rsi,
    fees=0.001,         # ví dụ phí 0.1% mỗi giao dịch
    slippage=0.0005,    # ví dụ trượt giá 0.05%
    direction='longonly'
)

stats_rsi = pf_rsi.stats()
print(stats_rsi)


Start Index                   2022-01-04 00:00:00+00:00
End Index                     2025-11-27 00:00:00+00:00
Total Duration                        973 days 00:00:00
Start Value                                       100.0
Min Value                                      37.44892
Max Value                                    102.545517
End Value                                      92.52761
Total Return [%]                               -7.47239
Benchmark Return [%]                           0.373134
Position Coverage [%]                         45.734841
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              63.480685
Max Drawdown Duration                 901 days 00:00:00
Total Orders                                          8
Total Fees Paid                                0.637513
Total Trades                                          4
Win Rate [%]                                       75.0
Best Trade [%]                                27

In [43]:
stats_rsi.index

Index(['Start Index', 'End Index', 'Total Duration', 'Start Value',
       'Min Value', 'Max Value', 'End Value', 'Total Return [%]',
       'Benchmark Return [%]', 'Position Coverage [%]',
       'Max Gross Exposure [%]', 'Max Drawdown [%]', 'Max Drawdown Duration',
       'Total Orders', 'Total Fees Paid', 'Total Trades', 'Win Rate [%]',
       'Best Trade [%]', 'Worst Trade [%]', 'Avg Winning Trade [%]',
       'Avg Losing Trade [%]', 'Avg Winning Trade Duration',
       'Avg Losing Trade Duration', 'Profit Factor', 'Expectancy',
       'Sharpe Ratio', 'Calmar Ratio', 'Omega Ratio', 'Sortino Ratio'],
      dtype='object')

In [33]:
# entries_rsi là SeriesFrame 1 cột bool: True = có entry
entry_ratio = entries_rsi.mean()

print("Entry ratio (RSI):", entry_ratio)
# Ví dụ: 0.05 nghĩa là ~5% số ngày có tín hiệu mua


Entry ratio (RSI): 0.018499486125385406


In [40]:
n_sims = 300
shape = (len(close), 1)

results = []

for seed in range(n_sims):
    # 4.1. Sinh random entry & exit dạng chain bằng generate_random_both
    entries_rand_df, exits_rand_df = pd.DataFrame.vbt.signals.generate_random_both(
        shape,
        entry_prob=entry_ratio,     # xác suất entry ~ tần suất RSI
        exit_prob=entry_ratio,      # xác suất exit
        seed=seed,

  
    )

    # Lấy Series 1 cột
    entries_rand = entries_rand_df
    exits_rand   = exits_rand_df

    # 4.2. Backtest random portfolio
    pf_rand = vbt.Portfolio.from_signals(
        close,
        entries=entries_rand,
        exits=exits_rand,
        fees=0.001,
        slippage=0.0005,
        direction='longonly'
    )

    st = pf_rand.stats()
    results.append({
        'seed': seed,
        'Total Return [%]': st['Total Return [%]'],
    
        'Sharpe Ratio': st['Sharpe Ratio']
    })

random_stats = pd.DataFrame(results)
print(random_stats.head())


   seed  Total Return [%]  Sharpe Ratio
0     0         48.905984      0.684427
1     1        114.269493      1.067196
2     2         -8.138869      0.012260
3     3         20.253009      0.403795
4     4         58.126997      0.828218


In [41]:

sharpe_rsi = stats_rsi['Sharpe Ratio']


sharpe_random = random_stats['Sharpe Ratio']


p_sharpe = (sharpe_random >= sharpe_rsi).mean()




print(f"Sharpe RSI: {sharpe_rsi:.2f}")
print(f"Sharpe Random mean: {sharpe_random.mean():.2f}")
print(f"RSI nằm trong top khoảng {100 * (1 - p_sharpe):.1f}% random theo Sharpe")


Sharpe RSI: 0.05
Sharpe Random mean: 0.14
RSI nằm trong top khoảng 43.0% random theo Sharpe
